# DBsimilarity — Basics in Chemometrics

Computational methods are now core to natural products research, especially when dealing with **unfractionated extracts** where signal overlap and chemical diversity complicate classic workflows. **Chemoinformatics** helps turn raw structures into actionable knowledge for:

* specimen prioritization,
* hypothesis generation for biological activity,
* biosynthetic and taxonomic inference,
* and rapid *in silico* dereplication.

## Goal of this notebook

Provide a clear, reproducible, end-to-end path that turns structure files into analysis-ready artifacts for MS/NMR dereplication and chemical space exploration—without requiring advanced programming skills.

## What you’ll do (overview)

1. **Convert** structural libraries (e.g., `.sdf`) to tidy **`.csv`** tables.
2. **Organize** data into a single Pandas **DataFrame** with consistent identifiers.
3. **Annotate** each SMILES with chemoinformatics fields (InChI, InChIKey, formula, exact mass, adduct m/z, SLogP).
4. **Build a custom MS database** (MZmine-style) for rapid **MS1 dereplication**.
5. **Assemble candidate lists for 2D NMR** dereplication (e.g., to support HSQC/TOCSY workflows).
6. **Quantify similarity** between compounds (Morgan fingerprints) and **cluster** using descriptors.
7. **Construct similarity networks** and export edge lists ready for **Cytoscape**.

Along the way you’ll also generate **MassQL** queries (MS1 and MS2) to search vendor-agnostic LC-MS(/MS) datasets, compute **Mordred** descriptors, run **hierarchical clustering** and **t-SNE** projections to visualize chemical space.

## Who this is for

Researchers who want practical, readable notebooks to:

* understand what’s in a specimen-specific compound list,
* create reusable MS/NMR dereplication assets,
* and explore chemical diversity with minimal coding.

## Inputs & outputs (at a glance)

* **Input:** structure files (`.sdf`) or tables (`.csv`) with at least names and SMILES.
* **Outputs:**

  * Annotated compound table (InChI, InChIKey, formula, exact mass, adduct m/z, SLogP)
  * MZmine custom DB CSV (MS1)
  * MassQL query files (MS1 and MS2)
  * Descriptor tables (Mordred) + dendrogram PNG + clustering CSV
  * t-SNE HTML plot (interactive)
  * Similarity network edge list + isolated nodes CSV (for Cytoscape)

## Minimal prerequisites

* **Python** (conda environment recommended)
* Core libraries: `pandas`, `numpy`, `matplotlib`, `scipy`, `scikit-learn`, `networkx`
* **RDKit** for structure parsing and fingerprints
* **Mordred** (optional) for extended descriptors
* **Plotly** (optional) for interactive t-SNE
* **Cytoscape** (optional) for network visualization

> Tip: Some RDKit/Mordred builds prefer `numpy<2`. If descriptor steps fail, pin NumPy accordingly.

## Learn more

* Pandas: [https://pandas.pydata.org/](https://pandas.pydata.org/)
* RDKit (Getting Started in Python): [https://www.rdkit.org/docs/GettingStartedInPython.html](https://www.rdkit.org/docs/GettingStartedInPython.html)
* ChEMBL web services: [https://chembl.gitbook.io/chembl-interface-documentation/web-services](https://chembl.gitbook.io/chembl-interface-documentation/web-services)
* Mordred: [http://mordred-descriptor.github.io/documentation/master/](http://mordred-descriptor.github.io/documentation/master/)


In [1]:
# --- Toggles (flip as you like) ---
USE_NBAGG          = True   # interactive matplotlib in Jupyter (fallbacks if unsupported)
ENABLE_PANDAS_TOOLS= True   # RDKit PandasTools (adds Molecule column helpers)
ENABLE_MORDRED     = True   # set False if not installed
ENABLE_CHEMBL      = False  # set True if you plan to query ChEMBL

# --- Stdlib & core libs ---
import os, sys, warnings, platform
from pathlib import Path
import numpy as np
import pandas as pd

# Pandas display niceties
pd.set_option("display.max_rows", 50)
pd.set_option("display.max_columns", 200)
pd.set_option("display.width", 180)
pd.set_option("display.float_format", lambda x: f"{x:.6g}")

# --- RDKit (don’t alias AllChem as Chem!) ---
try:
    from rdkit import RDLogger, Chem
    RDLogger.DisableLog('rdApp.*')          # silence verbose RDKit logs
    from rdkit.Chem import AllChem          # fingerprints, conformers
    from rdkit.Chem import Descriptors as RDDesc
    from rdkit.Chem.rdMolDescriptors import CalcMolFormula
    from rdkit import DataStructs
    RDKIT_OK = True
except Exception as e:
    RDKIT_OK = False
    print("⚠️ RDKit not available:", e)

# RDKit PandasTools (optional, controlled by toggle)
HAS_PANDAS_TOOLS = False
if ENABLE_PANDAS_TOOLS and RDKIT_OK:
    try:
        from rdkit.Chem import PandasTools
        HAS_PANDAS_TOOLS = True
    except Exception as e:
        print("ℹ️ RDKit PandasTools not available (continuing without it):", e)

# --- Mordred (optional) ---
if ENABLE_MORDRED:
    try:
        from mordred import Calculator, descriptors
        MORDRED_OK = True
    except Exception as e:
        MORDRED_OK = False
        print("⚠️ Mordred not available:", e)
else:
    MORDRED_OK = False

# --- Plotting ---
import matplotlib
if USE_NBAGG:
    try:
        matplotlib.use('nbagg')  # interactive zoom when supported
    except Exception:
        pass
import matplotlib.pyplot as plt
try:
    import seaborn as sns
    sns.set_context("notebook")
    sns.set_style("ticks")
except Exception:
    pass

# --- Plotly (optional, for interactive HTML like t-SNE) ---
try:
    import plotly
    PLOTLY_OK = True
except Exception as e:
    PLOTLY_OK = False
    print("ℹ️ Plotly not available (only affects interactive HTML exports):", e)

# --- Networks & misc ---
import networkx as nx
try:
    from wordcloud import WordCloud
except Exception:
    WordCloud = None

# --- ChEMBL client (optional) ---
if ENABLE_CHEMBL:
    try:
        from chembl_webresource_client.new_client import new_client
    except Exception as e:
        print("ℹ️ ChEMBL client not available (skipping):", e)
        new_client = None
else:
    new_client = None

# --- Project paths ---
PROJECT_ROOT = Path.cwd()
DATA_DIR     = PROJECT_ROOT / "data"
IMAGES_DIR   = PROJECT_ROOT / "images"
DATA_DIR.mkdir(exist_ok=True)
IMAGES_DIR.mkdir(exist_ok=True)

# --- Import your tools ---
# Prefer local project modules; fall back to the generic chem tools if present.
sys.path.append(str(PROJECT_ROOT))
sys.path.append(str(PROJECT_ROOT.parent))

dbs = None
ct  = None
try:
    import dbsimilarity_2 as dbs
    print("✅ Loaded dbsimilarity_2 as dbs")
except Exception as e:
    print("ℹ️ Could not import dbsimilarity_2:", e)
    try:
        import chem_tools_massql_similarity as ct
        print("✅ Loaded chem_tools_massql_similarity as ct")
    except Exception as e2:
        print("⚠️ Neither dbsimilarity_2 nor chem_tools_massql_similarity could be imported:", e2)

aliases = {
    "SMILES":   ["SMILES", "smiles", "***smiles***", "Smiles", "Smiles_parent"],
    "InChIKey": ["InChIKey", "INCHIKEY", "inchikey", "InChlKey", "InChI Key", "INCHI_KEY"],
    "Compound name": ["Compound name", "Name", "Compound", "compound_name"],
}        

# --- Quick environment summary ---
def env_summary():
    print("Python :", platform.python_version())
    print("Pandas :", pd.__version__)
    print("NumPy  :", np.__version__)
    if RDKIT_OK:
        try:
            from rdkit.rdBase import rdkitVersion
            print("RDKit  :", rdkitVersion)
        except Exception:
            print("RDKit  : OK (version not available)")
    else:
        print("RDKit  :", "missing")
    print("Mordred:", "OK" if MORDRED_OK else "missing/disabled")
    print("Plotly :", "OK" if PLOTLY_OK else "missing/disabled")
    print("PandasTools:", "OK" if HAS_PANDAS_TOOLS else "missing/disabled")

env_summary()

✅ Loaded dbsimilarity_2 as dbs
Python : 3.11.6
Pandas : 1.5.3
NumPy  : 1.23.5
RDKit  : 2022.09.5
Mordred: OK
Plotly : OK
PandasTools: OK


### Set the project name from the folder you’re working in

In [2]:
# Get the current working directory
current_directory = %pwd

# Extract the last folder from the path
last_folder = os.path.basename(current_directory)
Project = last_folder

Project

'Project_x'

### Load the target compound list and normalize identifiers

This cell reads your **target list** from `Target+".csv"` (semicolon-separated) and derives a canonical **InChIKey** from each SMILES. Having InChIKeys up front makes downstream matching, deduplication, and network building reliable and ID-agnostic.

Tips:

* Make sure the CSV has a **`SMILES`** column (and optionally a name/ID column).
* If your file uses a different delimiter or encoding, tweak `sep=";"` or add `encoding="utf-8"`.
* Invalid SMILES will raise errors later; consider filtering or reporting them here if needed.
* If you already have an `InchiKey` column, you can skip recomputing—or recompute to ensure consistency.


In [3]:
Target = "Actinobacteria"  # pass either a stem or filename with extension

path = dbs._detect_file(Target)
sep = dbs._guess_sep(path)
df_target, used_encoding = dbs._read_table(path, sep=sep)
smiles_col = dbs._normalize_smiles_column(df_target)

df_target = dbs.rename_by_aliases(df_target, aliases, inplace=False, prefer="max_notna")

dbs._add_inchikey(df_target, smiles_col="SMILES", out_col="InchiKey")

print(
    f"Loaded: {path.name} | sep='{sep}' | encoding='{used_encoding}' | "
    f"rows={df_target.shape[0]}, cols={df_target.shape[1]}"
)
invalid = df_target["InchiKey"].isna().sum()
if invalid:
    print(f"Note: {invalid} rows have invalid/unparseable SMILES (InchiKey = NaN).")

df_target.head(3)

Loaded: Actinobacteria.tsv | sep='	' | encoding='utf-8' | rows=8009, cols=35


,npaid,compound_id,Compound name,compound_molecular_formula,compound_molecular_weight,compound_accurate_mass,compound_m_plus_h,compound_m_plus_na,compound_inchi,compound_inchikey,SMILES,compound_cluster_id,compound_node_id,origin_type,genus,origin_species,original_reference_author_list,original_reference_year,original_reference_issue,original_reference_volume,original_reference_pages,original_reference_doi,original_reference_pmid,original_reference_title,original_reference_type,original_journal_title,synonyms_dois,reassignment_dois,synthesis_dois,mibig_ids,gnps_ids,cmmc_ids,npmrd_id,npatlas_url,InchiKey
0,NPA000003,3,A-503083 F,C18H22N4O13,502.389,502.118,503.126,525.107,InChI=1S/C18H22N4O13/c1-31-9-10(11(13(19)26)34...,RNRCUOCMUNIOMZ-UHFFFAOYSA-N,COC1C(C(OC1C(C(=O)N)OC2C(C(C=C(O2)C(=O)O)O)O)N...,3,3,Bacterium,Streptomyces,sp. SANK 62799,"Muramatsu, Yasunori; Ohnuki, Takashi; Ishii, M...",2004,10,57,639-646,10.7164/antibiotics.57.639,1.56383e+07,"A-503083 A, B, E and F, novel inhibitors of ba...",article,Journal of Antibiotics,[],[],[],['BGC0000288'],[],[],NP0005648,https://www.npatlas.org/explore/compounds/NPA0...,RNRCUOCMUNIOMZ-UHFFFAOYSA-N
1,NPA000006,6,Hibarimicin E,C77H100O34,1569.61,1568.61,1569.62,1591.6,InChI=1S/C77H100O34/c1-12-19-74(95)71(109-40-1...,QETQXVGQENIMIR-XYHXNMECSA-N,CCCC1(C(C(C(C2C1(C(=O)C3=C(C2)C=C4C(=C3O)C(=C(...,6,6,Bacterium,Microbispora,rosea subsp. hibaria,"CHO, SUNG IG; FUKAZAWA, HIDESUKE; HONMA, YOSHI...",2002,3,55,270-278,10.7164/antibiotics.55.270,1.20144e+07,Effects of Hibarimicins and Hibarimicin-Relate...,article,Journal of Antibiotics,[],[],[],[],[],[],NP0004164,https://www.npatlas.org/explore/compounds/NPA0...,QETQXVGQENIMIR-XYHXNMECSA-N
2,NPA000024,24,3097-B1,C15H21NO4,279.336,279.147,280.154,302.136,InChI=1S/C15H21NO4/c1-3-14(18)20-15-12(16-9-13...,HOTSKSXDTYBKBM-YDHLFZDLSA-N,CCC(=O)O[C@@H]1[C@H](CN[C@H]1CC2=CC=C(C=C2)OC)O,23,22,Bacterium,Streptomyces,sp. strain SA3097,"HOSOYA, YOSHIKO; KAMEYAMA, TOSHIYUKI; NAGANAWA...",1993,8,46,1300-1302,10.7164/antibiotics.46.1300,8.40759e+06,Anisomycin and new congeners active against hu...,article,Journal of Antibiotics,[],[],[],[],[],[],NP0022915,https://www.npatlas.org/explore/compounds/NPA0...,HOTSKSXDTYBKBM-YDHLFZDLSA-N


### Load the reference (“compounds of interest”) database and standardize IDs

This cell imports your curated reference set (e.g., CyanoMetDB / ChEMBL slice) from a semicolon-separated CSV encoded in **Latin-1**. It also fixes a common header typo by renaming **`InChlKey` → `InchiKey`**, so downstream joins and comparisons use a single, consistent identifier.

Why it matters:

* This table is your **search space** for dereplication and similarity checks against the target list.
* Consistent **InchiKey** naming avoids silent merge failures later.

Tips:

* After loading, consider trimming whitespace and **deduplicating by `InchiKey`** to prevent double counting.
* If you see weird characters, confirm the file’s encoding (utf-8 vs latin1) and delimiter (`sep=";"` here).
* Keep column names aligned across tables (`SMILES`, `InchiKey`, `Compound name`, etc.) for smooth merges.


In [4]:
Additional = "Actinobacteria"  # pass either a stem or filename with extension

path = dbs._detect_file(Additional)
sep = dbs._guess_sep(path)
df_Additional, used_encoding = dbs._read_table(path, sep=sep)
smiles_col = dbs._normalize_smiles_column(df_Additional)

df_Additional = dbs.rename_by_aliases(df_Additional, aliases, inplace=False, prefer="max_notna")

dbs._add_inchikey(df_Additional, smiles_col="SMILES", out_col="InchiKey")

print(
    f"Loaded: {path.name} | sep='{sep}' | encoding='{used_encoding}' | "
    f"rows={df_Additional.shape[0]}, cols={df_Additional.shape[1]}"
)
invalid = df_Additional["InchiKey"].isna().sum()
if invalid:
    print(f"Note: {invalid} rows have invalid/unparseable SMILES (InchiKey = NaN).")

df_Additional.head(3)

Loaded: Actinobacteria.tsv | sep='	' | encoding='utf-8' | rows=8009, cols=35


,npaid,compound_id,Compound name,compound_molecular_formula,compound_molecular_weight,compound_accurate_mass,compound_m_plus_h,compound_m_plus_na,compound_inchi,compound_inchikey,SMILES,compound_cluster_id,compound_node_id,origin_type,genus,origin_species,original_reference_author_list,original_reference_year,original_reference_issue,original_reference_volume,original_reference_pages,original_reference_doi,original_reference_pmid,original_reference_title,original_reference_type,original_journal_title,synonyms_dois,reassignment_dois,synthesis_dois,mibig_ids,gnps_ids,cmmc_ids,npmrd_id,npatlas_url,InchiKey
0,NPA000003,3,A-503083 F,C18H22N4O13,502.389,502.118,503.126,525.107,InChI=1S/C18H22N4O13/c1-31-9-10(11(13(19)26)34...,RNRCUOCMUNIOMZ-UHFFFAOYSA-N,COC1C(C(OC1C(C(=O)N)OC2C(C(C=C(O2)C(=O)O)O)O)N...,3,3,Bacterium,Streptomyces,sp. SANK 62799,"Muramatsu, Yasunori; Ohnuki, Takashi; Ishii, M...",2004,10,57,639-646,10.7164/antibiotics.57.639,1.56383e+07,"A-503083 A, B, E and F, novel inhibitors of ba...",article,Journal of Antibiotics,[],[],[],['BGC0000288'],[],[],NP0005648,https://www.npatlas.org/explore/compounds/NPA0...,RNRCUOCMUNIOMZ-UHFFFAOYSA-N
1,NPA000006,6,Hibarimicin E,C77H100O34,1569.61,1568.61,1569.62,1591.6,InChI=1S/C77H100O34/c1-12-19-74(95)71(109-40-1...,QETQXVGQENIMIR-XYHXNMECSA-N,CCCC1(C(C(C(C2C1(C(=O)C3=C(C2)C=C4C(=C3O)C(=C(...,6,6,Bacterium,Microbispora,rosea subsp. hibaria,"CHO, SUNG IG; FUKAZAWA, HIDESUKE; HONMA, YOSHI...",2002,3,55,270-278,10.7164/antibiotics.55.270,1.20144e+07,Effects of Hibarimicins and Hibarimicin-Relate...,article,Journal of Antibiotics,[],[],[],[],[],[],NP0004164,https://www.npatlas.org/explore/compounds/NPA0...,QETQXVGQENIMIR-XYHXNMECSA-N
2,NPA000024,24,3097-B1,C15H21NO4,279.336,279.147,280.154,302.136,InChI=1S/C15H21NO4/c1-3-14(18)20-15-12(16-9-13...,HOTSKSXDTYBKBM-YDHLFZDLSA-N,CCC(=O)O[C@@H]1[C@H](CN[C@H]1CC2=CC=C(C=C2)OC)O,23,22,Bacterium,Streptomyces,sp. strain SA3097,"HOSOYA, YOSHIKO; KAMEYAMA, TOSHIYUKI; NAGANAWA...",1993,8,46,1300-1302,10.7164/antibiotics.46.1300,8.40759e+06,Anisomycin and new congeners active against hu...,article,Journal of Antibiotics,[],[],[],[],[],[],NP0022915,https://www.npatlas.org/explore/compounds/NPA0...,HOTSKSXDTYBKBM-YDHLFZDLSA-N


#### Clean and deduplicate the reference table

This step removes unusable entries and redundancies before any joins or similarity work:

* Converts empty `SMILES` strings to `NaN` and **drops missing** structures.
* **Deduplicates** by `SMILES` to avoid double counting during stats, clustering, and network building.
* Prints the table size **before/after** so you can audit the impact.

Tips:

* If tautomers/salts matter, consider normalizing SMILES (`Chem.MolToSmiles(..., canonical=True)`) or deduplicating by **InChIKey** instead of raw SMILES.
* Optional: trim whitespace and standardize encoding before this step to catch edge cases.


In [5]:
# Filter row for empty cells
print(f"Size of the dataframe before filter empty cells: {df_Additional.shape}.")
df_Additional["SMILES"].replace('',np.nan,inplace=True)
df_Additional.dropna(subset="SMILES", inplace=True)
# Remove duplicate rows based on the "SMILES" column
df_Additional.drop_duplicates(subset=["SMILES"], inplace=True)
print(f"Size of the dataframe after filter empty and duplicated cells: {df_Additional.shape}.")

Size of the dataframe before filter empty cells: (8009, 35).
Size of the dataframe after filter empty and duplicated cells: (8008, 35).


### Merge target and reference, then flag the overlap (“compounds of interest”)

This cell unions the **target** list and the **reference** database on `InchiKey` using `merge_dataframes()`. That helper also adds presence flags:

* `additional_column_1` → present in **df\_target**
* `additional_column_2` → present in **df\_Additional_List**

We then extract the **intersection**—compounds that occur in **both** tables—as `df_Interest`, and print a quick audit of sizes so you can see how many candidates are immediately dereplicated.

Heads-up:

* Use parentheses when filtering to avoid precedence gotchas:

  ```python
  df_Interest = merged_df.loc[(merged_df['additional_column_1'] == 1) & (merged_df['additional_column_2'] == 1)]
  ```
* Consider renaming the flags after merge (e.g., `in_target`, `in_reference`) for readability.
* If stereochemistry/salt forms matter, `InchiKey` is a good join key; otherwise you may want to use standardized InChI or layer-specific keys depending on your workflow.


In [6]:
# Combine both dataframes
merged_df = dbs.merge_dataframes([df_target,df_Additional], 'InchiKey')
df_Interest = merged_df.loc[merged_df['additional_column_1'] & merged_df['additional_column_2'] == 1]
print(f"Size of the merged structure databases: df_target + df_Additional ({df_target.shape[0]+df_Additional.shape[0]}) = {merged_df.shape[0]}")
print(f"Size of the original structure database to study: {df_target.shape[0]}")
print(f"Size of the Additional structure database used as seed for compounds of interest: {df_Additional.shape[0]}")
print(f"How many compounds can be promptly identified as compounds of interest (co-occurring in both databases): {df_Interest.shape[0]}")

Size of the merged structure databases: df_target + df_Additional (16017) = 8007
Size of the original structure database to study: 8009
Size of the Additional structure database used as seed for compounds of interest: 8008
How many compounds can be promptly identified as compounds of interest (co-occurring in both databases): 8007


### Annotate structures with formula, exact mass, adduct m/z, InChI/InChIKey, and SLogP (customizable)

This cell runs the all-in-one annotator on `merged_df` and writes an extended table to `CyanosNP2010-2023_extended.csv`. You can toggle each computed field and choose data sources (Mordred vs RDKit for logP), ion mass convention (proton vs hydrogen atom), numeric precision, and which **adduct columns** to emit.

What you’ll get added (when enabled):

* `MolFormula`, `MolWeight` (neutral exact mass)
* Adduct m/z columns: `MolWeight-1H`, `MolWeight+1H`, `MolWeight+Na`, `MolWeight+K` (and optional `MolWeight+NH4`)
* `Inchi`, `InchiKey`
* `SLogP`

Notes & tips:

* `logp_source="mordred"` falls back to RDKit if Mordred isn’t available.
* `use_proton_mass=True` uses 1.007276466 for H⁺ / H⁻ (MS-friendly). Set `False` to use 1.007825032 (H atom).
* `drop_invalid_smiles=True` removes rows that can’t be parsed by RDKit (keeps outputs clean).
* Adjust `decimals` if you need more/less precision for m/z matching.


In [7]:
# Fully customized run
merged_df2 = dbs.annotate_smiles_table(
    merged_df,
    smiles_col="SMILES",
    compute_inchi=True,
    compute_inchikey=True,
    compute_formula=True,
    compute_exact_mass=True,
    compute_adduct_columns=True,
    compute_logp=True,
    logp_source="mordred",  # or "rdkit"
    # Your column names → adducts (you can add/remove as you like)
    adduct_columns={
        "MolWeight-1H": "[M-H]-",
        "MolWeight+1H": "[M+H]+",
        "MolWeight+Na": "[M+Na]+",
        "MolWeight+K":  "[M+K]+",
        # "MolWeight+NH4": "[M+NH4]+",    # uncomment if desired
    },
    use_proton_mass=True,    # False -> use hydrogen atom mass (1.007825032)
    decimals=6,
    drop_invalid_smiles=True,
    out_csv=f"{Project}_extended.csv")

### Generate MS1 MassQL queries (one per compound) and save to a text file

This cell builds **vendor-agnostic MassQL** queries that search MS1 spectra for each compound’s expected **adduct m/z** values (here: `[M+H]+`, `[M+Na]+`, `[M+K]+`, `[M+NH4]+`) within a **±ppm** tolerance and with a minimum **intensity percent**. It writes a numbered, human-readable list to `MS1_queries_by_compound.txt`, with one block per compound.

Notes & tips:

* `mass_col="MolWeight"` should be the **neutral monoisotopic mass**; adduct m/z are computed from it.
* Tune `ppm`, `intensity_percent`, and `decimals` to match your instrument and centroiding.
* Set `separate_adducts=True` if you prefer **one query per adduct** instead of an OR list.
* If names aren’t unique, consider prefixing with an internal ID to avoid ambiguity in the output.


In [8]:
# Build queries
q_dict = dbs.generate_massql_queries(
    merged_df2,
    ppm=10,
    intensity_percent=1,
    decimals=5,
    separate_adducts=False,
    adducts=["[M+H]+", "[M+Na]+", "[M+K]+", "[M+NH4]+"],
    name_col="Compound name",
    mass_col="MolWeight",)

out_path = Path("MS1_queries_by_compound.txt")
with out_path.open("w", encoding="utf-8") as f:
    for i, (name, q) in enumerate(q_dict.items(), start=1):
        f.write(f"### {i}. {name} ###\n{q}\n\n")  # blank line between queries


### Generate MS2 MassQL queries (precursor ± fragments) and save to a text file

This cell creates **MS2** queries that combine:

* a **precursor** constraint built from the neutral mass and the selected adducts (`[M+H]+`, `[M+Na]+`, `[M+K]+`, `[M+NH4]+`) with `ppm_prec`, and
* an optional **product-ion** list from the `Fragments` column matched with `ppm_prod` and an `INTENSITYPERCENT` floor.

Behavior & tips:

* If a row’s `Fragments` is empty, the query is **precursor-only** (still valid for MS2 filtering).
* Fragment values are cleaned (duplicates removed, non-numeric ignored), and zeros are dropped.
* **Cardinality**: by default, requires **1–5** fragment matches (clamped to available fragments). You can override with `cardinality_min` / `cardinality_max`.
* `decimals=4` controls how m/z values are rendered—adjust if your instrument resolution demands more/less precision.
* Tune `ppm_prec` vs `ppm_prod`: product ions often tolerate slightly tighter ppm than precursors, depending on your setup.

The result is written to `MS2_queries_by_compound.txt`, one numbered block per compound.


In [9]:
# Build
ms2_q = dbs.generate_massql_ms2_queries(
    merged_df2,
    name_col="Compound name",
    mass_col="MolWeight",        # or "Monoisotopic mass"
    fragments_col="Fragments",
    adducts=["[M+H]+","[M+Na]+","[M+K]+","[M+NH4]+"],
    ppm_prec=10,
    ppm_prod=10,
    cardinality_min= 1,
    cardinality_max= 5,
    intensity_percent=5,
    decimals=4)

# Write
out_path = Path("MS2_queries_by_compound.txt")
with out_path.open("w", encoding="utf-8") as f:
    for i, (name, q) in enumerate(ms2_q.items(), start=1):
        f.write(f"### {i}. {name} ###\n{q}\n\n")

KeyError: "Column 'Fragments' not found. Available: InchiKey, Compound name, SMILES, cmmc_ids, compound_accurate_mass, compound_cluster_id, compound_id, compound_inchi, compound_inchikey, compound_m_plus_h, compound_m_plus_na, compound_molecular_formula, compound_molecular_weight, compound_node_id, genus, gnps_ids, mibig_ids, npaid, npatlas_url, npmrd_id, origin_species, origin_type, original_journal_title, original_reference_author_list, original_reference_doi, original_reference_issue, original_reference_pages, original_reference_pmid, original_reference_title, original_reference_type, original_reference_volume, original_reference_year, reassignment_dois, synonyms_dois, synthesis_dois, additional_column_1, additional_column_2, MolWeight, MolFormula, Inchi, MolWeight-1H, MolWeight+1H, MolWeight+Na, MolWeight+K, SLogP"

### Prepare a clean submission table for NPClassifier

This cell exports the **minimal structure table** NPClassifier needs: a unique identifier (`InchiKey`) and the corresponding `SMILES`. The file is saved as `<Project>_for_NPClassifier.csv` with no index, ready to upload or batch process.

Tips:

* Consider deduplicating by `InchiKey` first to avoid repeated predictions.
* Make sure all `SMILES` parse in RDKit (invalid rows can be removed earlier).
* If you also want human-readable labels, you can include a `Compound name` column—extra columns are typically ignored by classifiers but remain useful for mapping results back.


In [10]:
# Constructing the .csv file to be submitted to NPClassifier 
df_NPclassifier0 = merged_df2[["InchiKey","SMILES"]]
df_NPclassifier0.to_csv(Project + '_for_NPClassifier.csv', index = False)

### Attach RDKit molecule objects (and fingerprints) to your table

This cell converts each `SMILES` into an RDKit **Molecule** and adds it as a new `Molecule` column (with embedded fingerprints because `includeFingerprints=True`). Many downstream ops (fingerprints, similarity, drawing) expect this column.

Notes & tips:

* This operation **modifies `merged_df2` in place**.
* It can be **memory-heavy** on large tables (molecules + fingerprints stored per row).
* Your comment mentions `SMILES_parent`, but the code uses `SMILES`. If the source column is actually `SMILES_parent`, change the call to:

  ```python
  PandasTools.AddMoleculeColumnToFrame(merged_df2, 'SMILES_parent', 'Molecule', includeFingerprints=True)
  ```
* If you only need molecules (no fingerprints), set `includeFingerprints=False` to save memory.


In [11]:
%%capture --no-display
# create the column 'Molecules' with the structures for each SMILES entree at the column 'SMILES_parent'
PandasTools.AddMoleculeColumnToFrame(merged_df2,'SMILES','Molecule',includeFingerprints=True)
print([str(x) for x in  merged_df2.columns])

### Compute an all-vs-all molecular similarity matrix (Morgan fingerprints)

This cell builds a **square similarity matrix** between every pair of structures using **Morgan fingerprints** (radius = 2, nBits = 2048 by default) and the **Dice** metric. Requirements:

* `merged_df2` must already contain an RDKit **`Molecule`** column and a unique **`InchiKey`** per row.
* The output `sim_df` has rows/columns indexed by `InchiKey`, with a **diagonal of 1.0**.

How to read/use it:

* Each entry ∈ \[0, 1] is a pairwise similarity; higher means more similar.
* You can switch to **Tanimoto** by passing `metric="tanimoto"`.
* Tune structure granularity with `radius` (2–3 are common) and control hash space via `nbits` (default 2048 in the helper).

Tips:

* Consider **deduplicating** molecules before running to avoid redundant rows.
* For large libraries, this is **O(n²)** in time/memory; filter first or compute similarities to a **subset/query set** if needed.


In [12]:
# Assuming merged_df already has RDKit Mol objects in 'Molecule' and an 'InchiKey' column:
sim_df = dbs.morgan_similarity_matrix(
                      merged_df2, 
                      mol_col="Molecule", 
                      id_col="InchiKey", 
                      radius=2, 
                      metric="dice")

display(sim_df.head(2))
print(f"SimTablet2 is the square matrix (with a diagonal = 1, {sim_df.shape}) with all the similarities calculated between every pair of structures.")

,AABZZWPMCAZHFC-UHFFFAOYSA-N,AANRCAZDPPXTKN-AXOLISODSA-N,AARICDGIAQPTML-RBFJYNKASA-N,AASPEXAITKEFPE-SEMUBUJISA-N,AATNFCIRDPNDRP-YRKVFUNNSA-N,AAUIGWGEXJDWSV-BJGZLQLRSA-N,AAUZLHJDJNJJDM-UHFFFAOYSA-N,AAVOQSXFLDIRRH-JUYSZZNHSA-N,AAYIADJXSCBLCS-GXLZLOGOSA-N,ABBFMUJAOCZWJG-GRZHNJOVSA-N,ABGLHEFIWWSNJQ-IRSNHEQCSA-N,ABHHIGWFFMCQOC-UHFFFAOYSA-N,ABLACSIRCKEUOB-UHFFFAOYSA-N,ABMDAGRSUWMYPO-AXUAAXHCSA-N,ABNGXYJNUFFHCJ-CNRHASOASA-N,ABNJCNGVZLNFJI-UHFFFAOYSA-N,ABNQIBOOQKGLRG-BPODRKPHSA-N,ABNUGWRZSYVEEW-INEUFUBQSA-N,ABOXJBHGJDKUNW-HBVXYWQDSA-N,ABTSKZKCMFRYNP-TXZMDZOJSA-N,ABYOOPVZTWKBKI-ZEULBBAFSA-N,ABZGUBNFEGMGPE-MQNVTSSDSA-N,ACLVPWVUGXKSQL-JVNMPXIPSA-N,ACLVPWVUGXKSQL-ZPFZUWPUSA-N,ACOBBFVLNKYODD-CSKARUKUSA-N,ACSDUXDWMODDMW-QWICZQNUSA-N,ACSULZLYEQGSAE-HMVSRFOSSA-N,ACTOXUHEUCPTEW-ZOTSFZJCSA-N,ACUGLGSAQKAJRT-UHFFFAOYSA-N,ACVGHZIZZPJLSQ-UHFFFAOYSA-N,ACVNJYFOTAGTNL-CCEZHUSRSA-N,ACYFBJUVNSGWDG-QGQKNWORSA-N,ACZKNHYYIFXIHX-UHFFFAOYSA-N,ADAGZDQSLWVWDS-OZNMWJIMSA-N,ADBVWUJBUGOHJH-UHFFFAOYSA-N,ADCDIHNCUQOKFP-XAIJYNPTSA-N,ADDJEJWGIYLORW-NGXZDTIWSA-N,ADDRECXFKLYOGC-MUAMBBPCSA-N,ADNJLVHPFULEDD-DBFOERGLSA-N,ADNWQRWIDDNCIH-VIFPVBQESA-N,ADSMKVUNHDDYBH-UHFFFAOYSA-N,ADUFSNUCLKTQTQ-RTHLEPHNSA-N,ADVPSQKLMVHHOI-MCKHRQRWSA-N,ADYJKPPPZQUXSH-UHFFFAOYSA-N,ADZBOAAUWYXFIU-SNVBAGLBSA-N,AEDRUJHNHSLWIZ-WTUOYXTGSA-N,AEDYCMURUGEFML-UHFFFAOYSA-N,AEFPQCAIXWXPSX-BZNITXMSSA-N,AEGYQCQXUUGBRT-ICFTUWFRSA-N,AEGZAZQDUFJYDZ-ALQHIDJXSA-N,AEHYYRHLFKWEDT-SZUFJOJHSA-N,AEIHQKDZICOEDE-JLBKHAGDSA-N,AEKIVMGNSYCSFZ-JTQLQIEISA-N,AEPMKZIOUKHDOO-MCIGGMRASA-N,AESYHMPUIGHGIN-ZFQJESHOSA-N,AETDFAXPCFASDB-UHFFFAOYSA-N,AETFMNMHSCDWEO-UHFFFAOYSA-N,AEULNKPXALBIHK-DXHYANOHSA-N,AEUPYRGOAQXDAN-CLJLJLNGSA-N,AEVLWFIASHAORK-DUKPNVDBSA-N,AEZPXPDCVAUXRV-CYGNKMLRSA-N,AFAFFSSNAUKMNO-YSTUSHMSSA-N,AFAXFXGTXQUJRU-GIZGSCRPSA-N,AFAZQKILLUAQTN-DIPKEJMWSA-N,AFBSECGAINOYAH-OQDLKAIYSA-N,AFDQPSWDNPFMNH-DTWKUNHWSA-N,AFDSDRVMDUIHOF-WLQWLBEYSA-N,AFDYEKHUMZAVTQ-KMYSFCLLSA-N,AFEUCARVKFDANA-NXEZZACHSA-N,AFFNBCRLIXERJL-UHFFFAOYSA-N,AFFNTHHWHDBPDH-UHFFFAOYSA-N,AFGBEDQMTFPVJS-NONQZKDXSA-N,AFGVKBBKHZVAKA-WHTXAGNKSA-N,AFGXGLIULLBHQO-LPHIXDDYSA-N,AFIGUHJLNOWSNK-DNMAVYRHSA-N,AFJZBVRODZOFAV-YRNVUSSQSA-N,AFMYMMXSQGUCBK-AKMKHHNQSA-N,AFOCLKRJYVPNLY-UHFFFAOYSA-N,AFOCUKZGFVPLEE-UHFFFAOYSA-N,AFQXBFSUWNVPRZ-UHFFFAOYSA-N,AFSHKCWTGFDXJR-SQOHEDJBSA-N,AGEJUJRSMSUTBD-JVEFKGINSA-N,AGGFQFOCZNGYPZ-ZDJUWLPYSA-N,AGGZTILVIJJOCM-CPTWIBRHSA-N,AGJUUQSLGVCRQA-DXJUKUTQSA-N,AGKYMAODIKREIE-UHFFFAOYSA-N,AGLMODUVVBZYHW-UHFFFAOYSA-N,AGNRGFRAFJOXHH-JGVFFNPUSA-N,AGPXPIZWMMLJPG-CBZXLTCLSA-N,AGRVSGSOQZGMAV-RKTNKWPKSA-N,AGSWTKCQYPFPKZ-ULNCKAKWSA-N,AHDUXXXZGSWYHF-IXGGKXOYSA-N,AHGKSZXKDPGMQU-IEBWSBKVSA-N,AHGUKZLDQLNILP-AAORUROXSA-N,AHHBHDOBNNJFAK-UHFFFAOYSA-N,AHHMIENJLSUKDM-INGGELRGSA-N,AHHRHPVDJMCSOB-UHFFFAOYSA-N,AHOIPAFUOXGGQB-IKNPUDIKSA-N,AHOSYNROBPSBAT-UHFFFAOYSA-N,AHPLMUVNCKCTFF-UHFFFAOYSA-N,...,ZTCKHTJXEMMWGV-CZUORRHYSA-N,ZTHCHEYYZNTCSW-DRPUZNPESA-N,ZTJJIBURSBTUAR-JKSUJKDBSA-N,ZTMDLJADCRURSE-KJXJVICLSA-N,ZTMRSAAMNPFCIE-UHFFFAOYSA-N,ZTQPYZKGVGYZEI-MIHBLRRSSA-N,ZTVHWWKOGFPEBH-STCRIONLSA-N,ZTVNUIIAPLINPK-NGJIGUQESA-N,ZTVQWZRGBFYBHB-UHFFFAOYSA-N,ZTZLJSSMDDCXOV-SSQPMERFSA-N,ZTZOKXOFRGHKSG-YPJQUURKSA-N,ZUADRGNPCRMAFH-CMEYIMQZSA-N,ZUCKJVVXVVOESY-ZSCLZSGASA-N,ZUCWNLVDTXGGSU-JTQLQIEISA-N,ZUDMXUJOZNUCIR-XRARMUINSA-N,ZUGQHQPKYXLART-UJBBIGQNSA-O,ZUGWRISHBCSTOJ-UHFFFAOYSA-N,ZUKYKBUZWIWCFL-UJEHSMEHSA-N,ZUMIOCAXBMIRDA-GDHIJPQJSA-N,ZUMYOZMKECONHU-UHFFFAOYSA-N,ZUPXAYGYALHVSA-VRZOSVEZSA-N,ZUPXAYGYALHVSA-WMLQCCHTSA-N,ZUTGGQMQIWDJCW-BCPYKWEZSA-N,ZUUILEHAHVINQF-DHDCSXOGSA-N,ZUUILEHAHVINQF-OQLLNIDSSA-N,ZUZCCBDSIBRLQK-UHFFFAOYSA-N,ZVCNREZZUJIBAG-UHFFFAOYSA-N,ZVEJGLKGBBJKBW-PKTZIBPZSA-N,ZVGNESXIJDCBKN-UUEYKCAUSA-N,ZVGNESXIJDCBKN-WUIGKKEISA-N,ZVHCTEUOWHUIMG-BTOYHFQMSA-N,ZVIQDPQQFMVLGU-XTCBYOQUSA-N,ZVJAKRLDGNGNOS-WGWGOIQKSA-N,ZVJOQUQNKVBWSE-OVBJUMGYSA-N,ZVLJONLPGJUTCW-UHFFFAOYSA-N,ZVOCJOZEMKTQEZ-ROHQPZTNSA-N,ZVQPCUCPQJKREF-YWEYNIOJSA-N,ZVRVZYXIMVTOQM-OAWIYRIWSA-N,ZVRXDSUQYITYNZ-UHFFFAOYSA-N,ZVWPMYHMXUXIMC-CBAPKCEASA-N,ZVWPMYHMXUXIMC-IONNQARKSA-N,ZVZPKUXZGROCDB-CKCDUWHISA-N,ZVZPKUXZGROCDB-QPTS

SimTablet2 is the square matrix (with a diagonal = 1, (8007, 8007)) with all the similarities calculated between every pair of structures.


### Build a similarity network, export edge list, and list isolated compounds

This cell turns the all-vs-all similarity matrix into a **network**:

* **Edges**: pairs with similarity **> 0.85** (strictly greater; equals are excluded by design).

  * Output: `links_filtered` (columns: `SOURCE`, `TARGET`, `CORRELATION`), deduplicated and undirected.
  * Saved as: `"<Project>_DB_compounds_Similarity_0.85.csv"` (semicolon-separated) if `save_csv=True`.
* **Graph**: `G` is a NetworkX graph built from the edge list; use it for quick stats or to write GraphML/GEXF for Cytoscape/Gephi.
* **Isolated nodes**: entries with **no edges above threshold** (i.e., chemically unique under current settings).

  * Output: `isolated_nodes_df`, saved to `"<Project>_isolated_nodes.csv"` when `save_isolated_csv=True`.

Notes & tips:

* If you want the identity column to be human-readable (e.g., `Compound name` or `file_path`), set `identity_col` accordingly. Right now it’s `InchiKey`.
* Make sure `metadata_df` points to the table that **contains** your identity column. If you annotated into `merged_df2`, pass `metadata_df=merged_df2`.
* To **include** pairs with similarity exactly equal to the cutoff, either slightly lower `threshold` (e.g., `0.8499`) or change the helper to use `>=`.
* Cytoscape import: load the edge CSV; choose `SOURCE`/`TARGET` as source/target and use `CORRELATION` as an edge attribute for styling or filtering.


In [13]:
links_filtered, G, isolated_nodes_df = dbs.build_similarity_network(
            sim_df,
            threshold=0.85,
            project=Project,
            save_csv=True,
            csv_sep=";",
            metadata_df=merged_df,       # or df_target if that’s where your identities live
            id_col="InchiKey",
            identity_col="InchiKey",    # change if your identity column is different
            save_isolated_csv=True)

# If you want to show InchiKey + file_path like you did:
cols = [c for c in ["InchiKey", "InchiKey"] if c in isolated_nodes_df.columns]
display(isolated_nodes_df[cols] if cols else isolated_nodes_df.head())

,InchiKey,InchiKey,InchiKey,InchiKey
0,AABZZWPMCAZHFC-UHFFFAOYSA-N,AABZZWPMCAZHFC-UHFFFAOYSA-N,AABZZWPMCAZHFC-UHFFFAOYSA-N,AABZZWPMCAZHFC-UHFFFAOYSA-N
1,AANRCAZDPPXTKN-AXOLISODSA-N,AANRCAZDPPXTKN-AXOLISODSA-N,AANRCAZDPPXTKN-AXOLISODSA-N,AANRCAZDPPXTKN-AXOLISODSA-N
3,AASPEXAITKEFPE-SEMUBUJISA-N,AASPEXAITKEFPE-SEMUBUJISA-N,AASPEXAITKEFPE-SEMUBUJISA-N,AASPEXAITKEFPE-SEMUBUJISA-N
5,AAUIGWGEXJDWSV-BJGZLQLRSA-N,AAUIGWGEXJDWSV-BJGZLQLRSA-N,AAUIGWGEXJDWSV-BJGZLQLRSA-N,AAUIGWGEXJDWSV-BJGZLQLRSA-N
12,ABLACSIRCKEUOB-UHFFFAOYSA-N,ABLACSIRCKEUOB-UHFFFAOYSA-N,ABLACSIRCKEUOB-UHFFFAOYSA-N,ABLACSIRCKEUOB-UHFFFAOYSA-N
...,...,...,...,...
8001,ZZFNBVPVZOECOF-UHFFFAOYSA-N,ZZFNBVPVZOECOF-UHFFFAOYSA-N,ZZFNBVPVZOECOF-UHFFFAOYSA-N,ZZFNBVPVZOECOF-UHFFFAOYSA-N
8002,ZZFQIJCZWAWDMW-LLXBHDTHSA-N,ZZFQIJCZWAWDMW-LLXBHDTHSA-N,ZZFQIJCZWAWDMW-LLXBHDTHSA-N,ZZFQIJCZWAWDMW-LLXBHDTHSA-N
8003,ZZIWTYGFYSRCOF-BXKUSQTMSA-N,ZZIWTYGFYSRCOF-BXKUSQTMSA-N,ZZIWTYGFYSRCOF-BXKUSQTMSA-N,ZZIWTYGFYSRCOF-BXKUSQTMSA-N
8004,ZZKFUHZBJMMIOO-UHFFFAOYSA-N,ZZKFUHZBJMMIOO-UHFFFAOYSA-N,ZZKFUHZBJMMIOO-UHFFFAOYSA-N,ZZKFUHZBJMMIOO-UHFFFAOYSA-N


### Export custom MS1 databases for MZmine (POS & NEG modes)

This cell creates two **custom MS1 database** CSVs compatible with MZmine:

* **POS** mode using the `[M+H]+` column (`MolWeight+1H`)
* **NEG** mode using the `[M−H]−` column (`MolWeight-1H`)

Each output has the columns (and order) MZmine expects:
`ID, m/z, Retention Time, identity, Formula`

What to know:

* `identity_col` should be a stable identifier you’ll recognize inside MZmine (here we use `InchiKey`; you could use `Compound name` or a filename path).
* `default_rt=0.0` sets a single retention time for all entries (fine for MS1 dereplication lists).
* Filenames default to `"<Project>_MZMine_CustomDB_POS.csv"` and `"<Project>_MZMine_CustomDB_NEG.csv"` in your working folder.

Tips:

* Ensure `MolWeight+1H` and `MolWeight-1H` already exist (use the annotator cell). These typically use the **proton mass** (1.007276466), which is standard for MS.
* You can refine later: add experiment-specific RTs per compound, or create separate DB files per subclass/taxon to speed matching.


In [16]:
# POS mode: [M+H]+
df_mzmine_pos = dbs.make_mzmine_custom_db(
        merged_df2,
        mz_col="MolWeight+1H",
        formula_col="MolFormula",
        identity_col="InchiKey",   # <— use a column that exists
        default_rt=0.0,
        project=Project,
        mode="POS",
        save_csv=True,)

# NEG mode: [M-H]-
df_mzmine_neg = dbs.make_mzmine_custom_db(
        merged_df2,
        mz_col="MolWeight-1H",
        formula_col="MolFormula",
        identity_col="InchiKey",
        default_rt=0.0,
        project=Project,
        mode="NEG",
        save_csv=True)

### Compute Mordred molecular descriptors (2D/3D) and save to CSV

This cell generates a wide set of **Mordred descriptors** for each SMILES and writes them to `"<Project>_DB_compounds_MordredDescriptors.csv"` (semicolon-separated). The return value `info` reports how many **2D**, **3D**, and **total** descriptors are available in your environment.

Notes & tips:

* `ignore_3D=False` requests **3D descriptors**; unless your molecules already have embedded 3D coordinates, many 3D fields may be **NaN**.

  * If you don’t need 3D (or don’t have conformers), set `ignore_3D=True` for a faster, denser table.
* If you *do* want 3D, generate conformers first (RDKit: `AllChem.EmbedMolecule` + `AllChem.UFFOptimizeMolecule`) and pass those mols to the function.
* Descriptor tables can be **large** (hundreds to thousands of columns). Plan memory accordingly.
* Some Mordred/RDKit builds prefer **NumPy < 2**; if you hit import/runtime issues, pin the version.

In [17]:
# MORDRED-Descriptors
df_descriptors, info = dbs.compute_mordred_descriptors(
        merged_df2,
        smiles_col="SMILES",
        ignore_3D=False,         # set True if you want 2D-only
        project=Project,         # used for filename
        save_csv=True)

2D:  1613
3D:   213
------------
total:  1826
C:\Users\borge\Documents\DBsimilarity.py\dbsimilarity_env\Lib\site-packages\numpy\core\fromnumeric.py:86: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
C:\Users\borge\Documents\DBsimilarity.py\dbsimilarity_env\Lib\site-packages\numpy\core\fromnumeric.py:86: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
C:\Users\borge\Documents\DBsimilarity.py\dbsimilarity_env\Lib\site-packages\numpy\core\fromnumeric.py:86: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
C:\Users\borge\Documents\DBsimilarity.py\dbsimilarity_env\Lib\site-packages\numpy\core\fromnumeric.py:86: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
C:\Users\borge\Documents\DBsimilarity.py\dbsimilarity_env\Lib\site-packages\numpy\core\fromnumeric.py:86: 

### Cluster compounds with a Ward dendrogram, save the figure, and subset by cluster

This cell performs **hierarchical clustering** on the Mordred descriptor matrix and draws a **Ward dendrogram**. The `color_threshold` both colors branch groups and—since `cut_distance=None`—acts as the **distance cut** to assign cluster IDs. With `run_agglomerative=True`, it also runs a matching **AgglomerativeClustering** to provide a complementary label set.

Outputs:

* **Figure** saved to `images/dendogram.png` (PNG) and displayed inline.
* **CSV** of cluster labels saved with a project-prefixed name.
* **`clusters`**: a `pd.Series` of hierarchical cluster IDs (you attach it to `merged_df2`).
* Example subset: all rows with `cluster == 3`.

Tips:

* Ensure `df_descriptors` is the numeric descriptor table produced earlier (rows aligned to your metadata).
* If you intended the standard spelling, change `filename="dendrogram.png"` to avoid confusion later.
* To control the number of clusters directly, lower `color_threshold` (tighter cut) or pass an explicit `cut_distance`.


In [ ]:
clusters, Z, fig, ax, df_clustered = dbs.dendrogram_and_cluster_descriptors(
        df_descriptors,                    # table from Mordred
        color_threshold=250000,
        xlim=(0, 700000),
        do_save_png=True,
        save_dir="images",
        filename="dendogram.png",
        cut_distance=None,                 # None -> uses color_threshold
        run_agglomerative=True,
        metadata_df=merged_df,             # to fetch file_path (if present)
        id_col="InchiKey",
        identity_col="file_path",
        project=Project,
        save_cluster_csv=True)
fig.show()

# If you want the same “cluster = 3” subset:
merged_df2["cluster"] = clusters  # if not already added via write-back
merged_dfX = merged_df2.loc[merged_df2["cluster"] == 3, ["cluster"]].dropna()

### Project the descriptor space to 2D with t-SNE and explore clusters interactively

This cell uses **t-SNE** to compress the high-dimensional Mordred descriptor space into **2D** for visual exploration. Points are **colored by the Agglomerative cluster labels** (`df_clustered["Clust"]`) and enriched with **hover info** (`file_path`, `cluster`, `Compound name`). The figure renders inline and is also saved as an interactive HTML at `images/t-sne.html` for sharing.

Tips:

* **Perplexity** must be sensible relative to sample size (rule of thumb: ≤ (n−1)/3). For small datasets, use 5–15; for larger, try 20–50.
* Set `standardize=True` if descriptor scales differ markedly.
* Results vary with initialization; add `random_state=0` for reproducibility if desired.
* If colors appear mismatched, ensure the **index** of `df_clustered` aligns with `df_descriptors`.


In [ ]:
# df_descriptors_values from the Mordred step (numeric only)
proj, fig = dbs.tsne_projection_plot(
        df_descriptors,                        # full DF; function picks numeric cols
        metadata_df=merged_df2,                 # for hover columns
        cluster_series=df_clustered["Clust"],  # color by sklearn cluster labels
        hover_cols=("file_path", "cluster", "Compound name"),
        standardize=False,                     # set True if scales differ a lot
        perplexity=20,
        out_html="images/t-sne.html"
)
fig.show()